In [1]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

params['dataset'] = 'nist'
#params['dataset'] = 'mnist'
params['model'] = 'mclr'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'] == 'mnist':
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['randomly'] = False
  params['allow_empty'] = True
  params['evenly'] = False
  params['num_group'] = 5
  params['min_clients'] = 2


"""
Common parameters
"""
params['num_rounds'] = 200
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20
params['seed'] = 233
params['agg_lr'] = 0.01


""" Set the output CSV file name """
info = '{}-{}'.format(params['optimizer'], params['dataset'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  if params['randomly'] == True:
    info2 = info2 + '-Randomly{:.1}-Evenly{:.1}'.format(str(params['randomly']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-nist-NumG5-Agglr0.01-ProximalF-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])

mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

200 Clients in Total


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Clustering Results: Counter({4: 27, 0: 20, 1: 18, 2: 18, 3: 17})
Clustering Inertia: 0.8652811243279721


In [4]:
server.train()

Training with 20 workers ---


Group 0, clients ['f_00154', 'f_00033', 'f_00074']
Group 1, clients ['f_00018', 'f_00098']
Group 2, clients ['f_00182', 'f_00012', 'f_00152', 'f_00061']
Group 3, clients ['f_00170', 'f_00180', 'f_00080', 'f_00130']
Group 4, clients ['f_00107', 'f_00177', 'f_00005', 'f_00146', 'f_00125', 'f_00007', 'f_00037']


Epoch:  35%|██████████████████████████▉                                                  | 7/20 [00:00<00:00, 68.71it/s]

Group 0
At round 0 accuracy: 0.36910994764397903
At round 0 training accuracy: 0.32910216718266255
At round 0 training loss: 2.3607565518872287
Group 1
At round 0 accuracy: 0.33568904593639576
At round 0 training accuracy: 0.3355676581483033
At round 0 training loss: 1.9621006213425092
Group 2
At round 0 accuracy: 0.32755298651252407
At round 0 training accuracy: 0.34668446817979526
At round 0 training loss: 2.4288669025096885
Group 3
At round 0 accuracy: 0.33047210300429186
At round 0 training accuracy: 0.33515121467526027
At round 0 training loss: 1.6519857785107184
Group 4
At round 0 accuracy: 0.31952662721893493
At round 0 training accuracy: 0.32191940368040994
At round 0 training loss: 2.2979635899088584
At round 0 mean test accuracy: 0.33523908523908524 mean train accuracy: 0.33371901833018697
The groups difference are: [0.         0.27416576 0.19306641 0.2942886  0.22554122]
Begin group  0 training


Epoch:  25%|███████████████████▎                                                         | 5/20 [00:00<00:00, 47.99it/s]

...

Begin group  1 training
Begin group  2 training


Epoch:  25%|███████████████████▎                                                         | 5/20 [00:00<00:00, 44.34it/s]

Begin group  3 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Begin group  4 training


In [5]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[4]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({7.0: 165, 6.0: 106, 8.0: 105, 9.0: 71, 5.0: 60})
507
